<a href="https://colab.research.google.com/github/morcellinus/2021_NH_BigData_Contest/blob/main/AVCD_%EC%B5%9C%EC%A2%85%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install exchange_calendars
import exchange_calendars as ecals 
import datetime
import pytz
import pandas as pd
import re
import numpy as np
import os
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, KFold
import imblearn
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

     |████████████████████████████████| 147 kB 7.3 MB/s 
  Created wheel for exchange-calendars: filename=exchange_calendars-3.4-py3-none-any.whl size=179566 sha256=5e318877aaa46fa91dce4fbab29b1d80da9e9105d4b3e34cba75e1a83dfd49a5
  Stored in directory: /root/.cache/pip/wheels/7c/ba/85/1819a0f65360ce0276efe87bc37a7c0681bcbedde4582c7ce0
Successfully built exchange-calendars


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
print(ecals.__version__)
print(pytz.__version__)
print(pd.__version__)
print(re.__version__)
print(np.__version__)
print(sklearn.__version__)
print(imblearn.__version__)
print(xgb.__version__)
print('개발환경 : Windows 10 버전 20H2')
# 버전 기재되지 않은 라이브러리는 버전이 따로 없는 라이브러리입니다. 

3.4
2018.9
1.1.5
2.2.1
1.19.5
0.22.2.post1
0.4.3
0.90
개발환경 : Windows 10 버전 20H2


In [ ]:
# 영업일 캘린더 불러오는 라이브러리 설치

XKRX = ecals.get_calendar("XKRX") # 한국 코드 설정


# 2016년 첫 영업일부터 2020년 12월 31일까지의 영업일만 남기기

xkrx_cal=XKRX.sessions_in_range(
    pd.Timestamp('20160104', tz=pytz.UTC),
   pd.Timestamp('20201231', tz=pytz.UTC))

In [ ]:
# 2016일 1월 4일부터 2020년 12월 31일까지의 한국 거래일 날짜들 데이터프레임으로 Data Leakage에 해당하지 않음

# 영업일 데이터를 데이터프레임 형태로 바꾸고 이 때 element는 string 형식이므로 정규표현식으로 형식 변환 후 수치형으로 변환

xkrx_df=xkrx_cal.to_frame(index=False, name='영업일')
xkrx_df['영업일']=xkrx_df['영업일'].dt.date
xkrx_df['영업일']=xkrx_df['영업일'].apply(lambda x: str(x))
xkrx_df['영업일']=xkrx_df['영업일'].apply(lambda x: re.sub("-","",x))
xkrx_df['영업일']=xkrx_df['영업일'].apply(lambda x: int(x))

In [ ]:
# numpy 시드 고정

np.random.seed(42)

# 데이터 불러오기
from google.colab import drive
drive.mount('/content/drive')
cus= pd.read_csv('/content/drive/Shareddrives/💰NH공모전💰/cus_info.csv', encoding = 'utf-8')
iem= pd.read_csv('/content/drive/Shareddrives/💰NH공모전💰/iem_info_20210902.csv', encoding = 'utf-8')
hist = pd.read_csv('/content/drive/Shareddrives/💰NH공모전💰/stk_bnc_hist.csv', encoding = 'utf-8')
train = pd.read_csv('/content/drive/Shareddrives/💰NH공모전💰/stk_hld_train.csv')
test = pd.read_csv('/content/drive/Shareddrives/💰NH공모전💰/stk_hld_test.csv')
submission = pd.read_csv('/content/drive/Shareddrives/💰NH공모전💰/sample_submission.csv', encoding = 'utf-8')

pd.set_option('display.max_row', 100)
pd.set_option('display.max_column', 100)

os.environ["PYTHONHASHSEED"] = str(42)

# Baseline code와 동일한 전처리 진행 (우선 Train data에 대해서만)

train_data = pd.merge(train, cus, how = "left", on = ["act_id"])
train_data = pd.merge(train_data, iem, how = "left", on = ["iem_cd"])

hist["stk_p"] = hist["tot_aet_amt"] / hist["bnc_qty"]
hist = hist.fillna(0)

train_data = pd.merge(train_data, hist, how = "left", on = ["act_id", "iem_cd"])
train_data = train_data[(train_data["byn_dt"] == train_data["bse_dt"])]
train_data.reset_index(drop = True, inplace = True)

Mounted at /content/drive


In [ ]:
# byn_dt + hold_d만큼의 판매날짜를 계산해주는 함수 정의

def find_day(df,idx, hold_d):

  # 보유일자 10일; 6월 1일에 구매했다 : 6월 10일에 판매했다
  
    new=idx+hold_d-1
    return df.iloc[new,0]

In [ ]:
######################################
#  이 과정이 최소 5시간 정도 소요됨  #
#         따라서 완성된 df를         #
# dacon@dacon.io 계정으로 보냈습니다 #
######################################

################################################
#긴 시간동안 코드가 돌아가므로 주석처리했습니다#
################################################

# 함수 실행하여 구매일자부터 보유일자만큼 값을 더해 판매일자를 구한 데이터프레임 생성
'''
train_data['판매날짜']=''

for j in range(len(train_data)):
    
    # day = byn_dt
    day=train_data.iloc[j,2]

    # hold_d = hold_d
    hold_d=train_data.iloc[j,3]

    # 영업일 데이터에서 byn_dt와 일치하는 영업일의 인덱스 추출
    idx=xkrx_df[xkrx_df['영업일']==day].index

    # train_data에 판매날짜라는 column을 만들고 구매날짜(byn_dt)와 일치하는 영업일의 인덱스 + hold_d - 1만큼의 인덱스의 날짜를 판매한 날짜로 추가

    train_data.loc[j,'판매날짜']=find_day(xkrx_df,idx, hold_d).values


# 판매날짜가 추가된 train_data 앞으로 판매_train이라 지칭

판매_train = train_data
'''

In [ ]:
# 따로 보내드린 DF를 사용해 바로 이전 셀을 건너뛰시고 아래 코드만 실행하시면 됩니다.
import pandas as pd
판매_train = pd.read_csv('/content/drive/Shareddrives/💰NH공모전💰/판매날짜_train_보유일수정.csv') 

In [ ]:
판매_train

,act_id,iem_cd,byn_dt,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p,판매날짜
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11,1,9,3,2,2,9,5,5,8,GS건설,1,1,1,20180726,274.0,11782000.0,5000.0,43000.0,20180809
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1,20180131,2.0,4990000.0,5000.0,2495000.0,20180530
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5,1,9,3,2,2,9,5,5,8,코스모신소재,12,2,99,20180517,786.0,14619600.0,1000.0,18600.0,20180524
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22,1,9,3,2,2,9,5,5,8,영진약품,8,2,1,20201112,60.0,462000.0,500.0,7700.0,20201211
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324,1,9,3,2,2,9,5,5,8,아세아제지,10,3,99,20180905,183.0,8125200.0,5000.0,44400.0,20200102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681467,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A260660,20180831,1,1,4,4,2,2,3,4,3,8,알리코제약,10,3,99,20180831,199.0,3532250.0,500.0,17750.0,20180831
681468,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A271980,20201027,1,1,4,4,2,2,3,4,3,8,제일약품,8,2,99,20201027,488.0,22960400.0,500.0,47050.0,20201027
681469,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A289080,20181121,1,1,4,4,2,2,3,4,3,8,SV인베스트먼트,2,2,99,20181121,2210.0,9204650.0,500.0,4165.0,20181121
681470,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A307930,20200214,1,1,4,4,2,2,3,4,3,8,컴퍼니케이,2,3,99,20200214,285.0,2721750.0,500.0,9550.0,20200214


In [ ]:
# train에서 구매년도와 판매년도 뽑아내기
# 현재 구매년도와 판매년도 컬럼은 YYYYMMDD형식의 수치형 변수이므로 10000으로 나누면 그 몫인 YYYY만 도출 가능

판매_train['구매년도']=판매_train['byn_dt']//10000
판매_train['판매년도']=판매_train['판매날짜']//10000

In [ ]:
# 이후 오버샘플링을 위한 가짜 종속변수 생성
# 판매년도가 구매년도보다 뒤인 경우, 즉 Test data의 관측값들과 같은 경우 가짜 종속변수 값으로 1을 주고 나머지 경우에 0을 할당

판매_train['fake_y'] = np.where(판매_train['구매년도']<판매_train['판매년도'], 1, 0)

In [ ]:
# test_data가 구매년도와 판매년도가 다른 데이터였으므로, 최대한 test_data와 분포가 비슷하도록 구매년도와 판매년도가 다른 경우만 있도록 데이터를 새로 구성함
# 이때, test_d에서의 hist_d는 21년 판매하는 종목에 대해 구매일자 ~ 20년 12월 30일의 보유기간을 나타내므로
# train_data에서 hist_d 또한 구매년도와 판매년도가 다른 데이터에서 (ex) 16년 구매, 20년 판매 : 구매일자~ 19년 마지막 영업일 보유기간을 hist_d로 설정함

# 영업일에 따른 판매날짜 계산을 위한 함수 생성
# 각 해마다 영업일이 다르므로 해마다 따로 함수를 정의

def find_histd17(day): #16년도
  idx=xkrx_df[xkrx_df['영업일']==day].index
  idx=idx.values
  hist_d=245-idx #245는 xkrx_df에서 2016년 마지막 영업일의 인덱스값
  return hist_d

def find_histd18(day): #17년도
  idx=xkrx_df[xkrx_df['영업일']==day].index
  idx=idx.values
  hist_d=488-idx #488은 xkrx_df에서 2017년 마지막 영업일의 인덱스값
  return hist_d

def find_histd19(day): #18년도
  idx=xkrx_df[xkrx_df['영업일']==day].index
  idx=idx.values
  hist_d=732-idx #732은 xkrx_df에서 2018년 마지막 영업일의 인덱스값
  return hist_d

def find_histd20(day): # 19년도
  idx=xkrx_df[xkrx_df['영업일']==day].index
  idx=idx.values
  hist_d=978-idx #978은 xkrx_df에서 2019년 마지막 영업일의 인덱스값
  return hist_d

# 각 판매년도에 따라 hist_d 값 계산

target_17=판매_train[판매_train['판매년도']==2017]
target_17['hist_d']=''
target_17['hist_d']=target_17['byn_dt'].apply(lambda x: find_histd17(x))
target_17['hist_d'] = target_17['hist_d'].astype(str).str.replace(r"[^0-9]","").astype(int)

target_18=판매_train[판매_train['판매년도']==2018]
target_18['hist_d']=''
target_18['hist_d']=target_18['byn_dt'].apply(lambda x: find_histd18(x))
target_18['hist_d'] = target_18['hist_d'].astype(str).str.replace(r"[^0-9]","").astype(int)

target_19=판매_train[판매_train['판매년도']==2019]
target_19['hist_d']=''
target_19['hist_d']=target_19['byn_dt'].apply(lambda x: find_histd19(x))
target_19['hist_d'] = target_19['hist_d'].astype(str).str.replace(r"[^0-9]","").astype(int)

target20=판매_train[판매_train['판매년도']==2020]
target20['hist_d']=''
target20['hist_d']=target20['byn_dt'].apply(lambda x: find_histd20(x))
target20['hist_d'] = target20['hist_d'].astype(str).str.replace(r"[^0-9]","").astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
# 위에서 생성한 판매년도 별 데이터 합친 후 인덱스 정리

targets=pd.concat([target20,target_17,target_18,target_19])
targets.reset_index(drop=True,inplace=True)

# 변수명 변경

train_data = targets

In [ ]:
# Test data에 대한 전처리 및 Train과 Test data 자연어 라벨 인코딩

test_data = pd.merge(test, cus, how = "left", on = ["act_id"])
test_data = pd.merge(test_data, iem, how = "left", on = ["iem_cd"])

hist["stk_p"] = hist["tot_aet_amt"] / hist["bnc_qty"]
hist = hist.fillna(0)

test_data = pd.merge(test_data, hist, how = "left", on = ["act_id", "iem_cd"])
test_data = test_data[(test_data["byn_dt"] == test_data["bse_dt"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["act_id", "iem_cd", "byn_dt", "bse_dt"], axis = 1)
test_data = test_data.drop(["act_id", "iem_cd", "byn_dt", "submit_id", "hold_d", "bse_dt"], axis = 1)


In [ ]:
L_encoder = LabelEncoder()
L_encoder.fit(iem["iem_krl_nm"])
train_data["iem_krl_nm"] = L_encoder.transform(train_data["iem_krl_nm"])
test_data["iem_krl_nm"] = L_encoder.transform(test_data["iem_krl_nm"])
train_data = train_data.drop(['판매날짜', '구매년도', '판매년도'], axis = 1)

In [ ]:
# 오버 샘플링

# SMOTE oversampling 파라미터 설정, K값을 10으로 설정

smote = SMOTE(random_state = 2021, k_neighbors = 10)

x_train = train_data.drop(['fake_y'], axis = 1)
y_train = train_data['fake_y']
x_train_st, y_train_st = smote.fit_resample(x_train, y_train)

# 오버 샘플링 결과

print("Train Dataset shape {} / {}".format(x_train_st.shape, y_train_st.shape))

In [ ]:
# 오버 샘플링 결과에 column명 지정해주기 

st_x = pd.DataFrame(x_train_st, columns = x_train.columns).astype(int)
st_y = pd.DataFrame(y_train_st, columns = ['y']).astype(object)

# 오버 샘플링 결과로 만든 독립변수와 종속변수 병합

st = pd.concat([st_x, st_y], axis = 1)

# 가짜 종속변수가 1인 row (판매년도<구매년도인 경우)만 남기고 가짜 종속변수 삭제

train_long = st[st['y']==1]
train_long = train_long.drop('y', axis = 1)

In [ ]:
# Train data 열 순서 정리

train_long = train_long[['hist_d', 'sex_dit_cd', 'cus_age_stn_cd', 'ivs_icn_cd', 'cus_aet_stn_cd', 'mrz_pdt_tp_sgm_cd','lsg_sgm_cd', 'tco_cus_grd_cd', 'tot_ivs_te_sgm_cd', 'mrz_btp_dit_cd', 'iem_krl_nm', 'btp_cfc_cd', 'mkt_pr_tal_scl_tp_cd', 'stk_dit_cd', 'bnc_qty', 'tot_aet_amt', 'stk_par_pr', 'stk_p', 'hold_d']]

# Train_data row 순서 정리

train_long = train_long.reset_index(drop = False, inplace = False)
train_long = train_long.drop('index', axis = 1)

NameError: ignored

In [ ]:
# XGBoost 모델링

train_data = train_long.drop('hold_d', axis = 1)
train_label = train_long['hold_d']

models = []

# 5-fold CV

folds = KFold(n_splits = 5, random_state = 42)

for train_idx, val_idx in folds.split(train_data):

  train_x = train_data.iloc[train_idx,:]
  train_y = train_label[train_idx]
  val_x = train_data.iloc[val_idx, :]
  val_y = train_label[val_idx]

  model = XGBRegressor(learning_rate = 0.01,
                       max_depth = 5,
                       min_child_weight = 2,
                       n_estimators = 1000,
                       subsample = 0.6,
                       alpha = 1,
                       random_state = 42)
  
  model.fit(train_x, train_y,
            eval_set = [(val_x, val_y)],
            eval_metric = ['rmse'])
  
  model.fit(train_data, train_label)

  models.append(model)

In [ ]:
# Test data 적합

result = []

for i in models:
  result.append(i.predict(test_data))
predict = np.mean(result, axis = 0)

In [ ]:
# Submission Data 생성

submission['hold_d'] = np.round(predict)
submission.to_csv('/content/drive/Shareddrives/💰NH공모전💰/Final_submission.csv', index = False)